In [ ]:
# Import required libraries
import pandas_datareader as pdr
import math


import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

import yfinance as yf
yf.pdr_override()
from pandas_datareader import data as pdr
# Define start and end dates
end_date = '2018-01-01'
start_date = '2001-01-01'

# Get the close prices for GBPUSD
df_gbpusd = pdr.get_data_yahoo('GBPUSD=X', start=start_date, end=end_date)
data_gbpusd = df_gbpusd.values
data_gbpusd = np.diff(data_gbpusd)
data_gbpusd2 = np.diff(df_gbpusd.filter(['Close']).values, axis=0)

# Get the close prices for SPX500
df_spx500 = pdr.get_data_yahoo('^GSPC', start=start_date, end=end_date)
data_spx500 = df_spx500.values
data_spx500 = np.diff(data_spx500, axis=0)

# Find the minimum length between the two datasets
min_length = min(len(data_gbpusd), len(data_spx500))

# Truncate the longer dataset to match the minimum length
data_gbpusd = data_gbpusd[:min_length]
data_spx500 = data_spx500[:min_length]

# Take every 7th element from the truncated datasets
data_gbpusd = data_gbpusd[::]
data_spx500 = data_spx500[::]




# Get the close prices for USDMXN (MXN=X)
df_usdmxn = pdr.get_data_yahoo('MXN=X', start=start_date, end=end_date)
data_usdmxn = df_usdmxn.values
data_usdmxn = np.diff(data_usdmxn, axis=0)
data_usdmxn = data_usdmxn[:min_length]
data_usdmxn = data_usdmxn[::]

# Get the close prices for USDCAD (CAD=X)
df_usdcad = pdr.get_data_yahoo('CAD=X', start=start_date, end=end_date)
data_usdcad = df_usdcad.values
data_usdcad = np.diff(data_usdcad, axis=0)
data_usdcad = data_usdcad[:min_length]
data_usdcad = data_usdcad[::]

# Get the close prices for USDCNY (CNY=X)
df_usdcny = pdr.get_data_yahoo('CNY=X', start=start_date, end=end_date)
data_usdcny = df_usdcny.values
data_usdcny = np.diff(data_usdcny, axis=0)
data_usdcny = data_usdcny[:min_length]
data_usdcny = data_usdcny[::]

# Get the close prices for VIX (%5EVIX)
df_vix = pdr.get_data_yahoo('%5EVIX', start=start_date, end=end_date)
data_vix = df_vix.values
data_vix = np.diff(data_vix, axis=0)
data_vix = data_vix[:min_length]
data_vix = data_vix[::]
def data(dataName):
  df_data = pdr.get_data_yahoo(dataName, start=start_date, end=end_date)
  data_data = df_data.values
  data_data = np.diff(data_data, axis=0)
  data_data = data_data[:min_length]
  data_data = data_data[::]
  return data_data
# Concatenate all the lists
lists = [data_spx500, data_usdmxn, data_usdcad, data_usdcny, data_vix,
         data('EURUSD%3DX'), data('AUDUSD%3DX'), data('NZDUSD%3DX'),
         data('EURJPY%3DX'), data('GBPJPY%3DX'), data('EURGBP%3DX'), data('EURCAD%3DX'),
         data('EURSEK%3DX'), data('EURCHF%3DX'), data('%5ETNX'), data('%5EDJI'),
         data('%5EIXIC'), data('CL%3DF'), data('GC%3DF'), data('%5EFVX'), data('%5ETYX'),
         data('%5EFTSE'), data('%5ENYA'), data('%5EXAX'), data('%5ERUT'), data('%5EGDAXI'),
         data('%5EFCHI'), data('%5ESTOXX50E'), data('%5EN100'), data('%5EBFX'),
         data('%5EN225'), data('%5EHSI'), data('000001.SS'), data('399001.SZ'), data('%5ESTI'),
         data('%5EAXJO'), data('%5EAORD'), data('%5EBSESN'), data('%5EJKSE'), data('%5EKLSE'),
         data('%5ENZ50'), data('%5EKS11'), data('%5ETWII'), data('%5EGSPTSE'), data('%5EBVSP'),
         data('%5EMXX'), data('%5EIPSA'), data('%5EMERV'), data('%5ETA125.TA'), data('%5EIRX')]


# Find the maximum length among all lists
max_length = max(len(lst) for lst in lists)

# Resize lists to have the maximum length
resized_lists = [np.resize(lst, (max_length, 1)) for lst in lists]

# Concatenate the resized lists
data_combined = np.concatenate(resized_lists, axis=1)
"""
data_combined = np.concatenate((data_gbpusd, data_spx500, data_usdmxn, data_usdcad, data_usdcny, data_vix, data('EURUSD%3DX'), data('AUDUSD%3DX'), data('NZDUSD%3DX'), data('EURJPY%3DX'), data('GBPJPY%3DX'), data('EURGBP%3DX'), data('EURCAD%3DX'), data('EURSEK%3DX'), data('EURCHF%3DX'),  data('%5ETNX'), data('%5EDJI'), data('%5EIXIC'), data('CL%3DF'), data('GC%3DF'), data('%5EFVX'), data('%5ETYX'), data('%5EFTSE'), data('%5ENYA'), data('%5EXAX'), data('%5ERUT'), data('%5EGDAXI'), data('%5EFCHI'), data('%5ESTOXX50E'), data('%5EN100'), data('%5EBFX'), data('%5EN225'), data('%5EHSI'), data('000001.SS'), data('399001.SZ'), data('%5ESTI'), data('%5EAXJO'), data('%5EAORD'), data('%5EBSESN'), data('%5EJKSE'), data('%5EKLSE'), data('%5ENZ50'), data('%5EKS11'), data('%5ETWII'), data('%5EGSPTSE'), data('%5EBVSP'), data('%5EMXX'), data('%5EIPSA'), data('%5EMERV'), data('%5ETA125.TA')), axis=1)
"""
# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data_combined)
scalegbpusd = scaler.fit_transform(data_gbpusd2)
# Define training data length
training_data_len = math.ceil(len(data_combined) * .8)

# Create the training data set
train_data = scaled_data[0:training_data_len, :]
scalegbpusd = scalegbpusd[0:training_data_len, :]
# Split data into x_train and y_train
x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i - 60:i, :])
    y_train.append(scalegbpusd[i, 0])

# Convert x_train and y_train to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

# Reshape x_train for LSTM input
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 50))
"""
lastnum = scaler.inverse_transform(model.predict(x_train))[-1]
lastone = (lastnum)**2
lastdata = data_gbpusd[-1]**2
print(lastone - lastdata)
print(lastnum - data_gbpusd[-1])
"""

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

'\nlastnum = scaler.inverse_transform(model.predict(x_train))[-1]\nlastone = (lastnum)**2\nlastdata = data_gbpusd[-1]**2\nprint(lastone - lastdata)\nprint(lastnum - data_gbpusd[-1])\n'

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import GRU
# Build the model
model = Sequential()
model.add(GRU(500, return_sequences=True, input_shape=(x_train.shape[1], 50)))
model.add(GRU(250, return_sequences=False))
model.add(Dense(200))
model.add(Dense(150))
model.add(Dense(100))
model.add(Dense(75))
model.add(Dense(50))
model.add(Dense(25))
model.add(Dense(1))



# Compile the model
print('Done adding')
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[tf.keras.metrics.Accuracy()])


Done adding


In [ ]:
import keras
from keras.models import Model
from tensorflow.keras.models import load_model


# Load the model from the SavedModel format
model = keras.models.load_model('Delta.h5')

In [ ]:
model.fit(x_train, y_train, batch_size=3, epochs=150)

Epoch 1/150
956/956 [==============================] - 389s 407ms/step - loss: 0.0036 - accuracy: 0.0000e+00
Epoch 2/150
956/956 [==============================] - 387s 405ms/step - loss: 0.0037 - accuracy: 0.0000e+00
Epoch 3/150
956/956 [==============================] - 387s 405ms/step - loss: 0.0037 - accuracy: 0.0000e+00
Epoch 4/150
945/956 [============================>.] - ETA: 4s - loss: 0.0037 - accuracy: 0.0000e+00

KeyboardInterrupt: ignored

In [ ]:
new = model.predict(x_train)

125/125 [==============================] - 26s 207ms/step


In [ ]:
print(type(new))

<class 'numpy.ndarray'>


In [ ]:
np.set_printoptions(threshold=np.inf)
new2 = new
new3 = np.diff(new2, axis=0)
new4 = np.diff(new3, axis=0)
print(new3[:-60])

[[ 5.30093908e-04]
 [ 1.31946802e-03]
 [-1.21742487e-04]
 [-2.60621309e-04]
 [-1.09374523e-04]
 [-6.84946775e-04]
 [-4.32282686e-04]
 [-1.96963549e-04]
 [-1.76131725e-05]
 [-8.14497471e-04]
 [-7.59661198e-04]
 [ 1.59382820e-04]
 [-4.00543213e-05]
 [-2.02298164e-04]
 [-6.46293163e-04]
 [-1.97380781e-04]
 [-1.78813934e-05]
 [ 1.75893307e-04]
 [-2.69442797e-04]
 [-2.64674425e-04]
 [ 3.38137150e-04]
 [-1.44898891e-04]
 [-3.51876020e-04]
 [ 3.62128019e-04]
 [-3.13222408e-04]
 [ 1.41859055e-05]
 [-3.43948603e-04]
 [ 1.60813332e-04]
 [ 2.62856483e-04]
 [ 4.20868397e-04]
 [-2.38418579e-05]
 [-5.13315201e-04]
 [-2.16066837e-05]
 [ 4.30107117e-04]
 [ 4.16994095e-04]
 [ 1.88142061e-04]
 [ 1.22070312e-03]
 [ 8.28862190e-04]
 [ 1.90019608e-04]
 [-2.93582678e-04]
 [-3.03983688e-04]
 [-5.60849905e-04]
 [-3.33338976e-04]
 [-6.29425049e-05]
 [-2.93850899e-05]
 [-2.12460756e-04]
 [-1.90436840e-04]
 [ 7.80820847e-05]
 [-3.71456146e-04]
 [ 1.75774097e-04]
 [ 4.39852476e-04]
 [ 1.02809072e-03]
 [ 7.8713893

In [ ]:
import numpy as np

# Define the two arrays

# Calculate the percentage similarity
similarity = np.mean(new2 == np.diff(scalegbpusd[60:])) * 100

# Print the percentage similarity
print(f"Percentage similarity: {similarity}%")


Percentage similarity: nan%


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
print(len(np.diff(scalegbpusd[60:])))
print(len(new4))

3993
5006


In [ ]:
model.fit(x_train, y_train, batch_size=3, epochs=1)

956/956 [==============================] - 177s 185ms/step - loss: 5.5572e-05 - accuracy: 0.0000e+00


In [ ]:
model.fit(x_train, y_train, batch_size=3, epochs=15)

Epoch 1/15
1332/1332 [==============================] - 383s 285ms/step - loss: 0.0011 - accuracy: 0.0000e+00
Epoch 2/15
1332/1332 [==============================] - 374s 281ms/step - loss: 1.7741e-04 - accuracy: 0.0000e+00
Epoch 3/15
1332/1332 [==============================] - 376s 282ms/step - loss: 1.3741e-04 - accuracy: 0.0000e+00
Epoch 4/15
1332/1332 [==============================] - 380s 285ms/step - loss: 9.5744e-05 - accuracy: 0.0000e+00
Epoch 5/15
1332/1332 [==============================] - 372s 280ms/step - loss: 1.9755e-04 - accuracy: 0.0000e+00
Epoch 6/15
1332/1332 [==============================] - 374s 281ms/step - loss: 8.2179e-05 - accuracy: 0.0000e+00
Epoch 7/15
1332/1332 [==============================] - 377s 283ms/step - loss: 9.2141e-05 - accuracy: 0.0000e+00
Epoch 8/15
1332/1332 [==============================] - 380s 285ms/step - loss: 1.5602e-04 - accuracy: 0.0000e+00
Epoch 9/15
1332/1332 [==============================] - 380s 285ms/step - loss: 8.9043e-05 -

In [ ]:
model.save('Delta2.h5')

In [ ]:
list = model.predict(x_train)

125/125 [==============================] - 21s 165ms/step


In [ ]:
print(list[-100:])

[[0.46110156]
 [0.4618495 ]
 [0.44923282]
 [0.43497056]
 [0.44630268]
 [0.44285956]
 [0.4325801 ]
 [0.42046058]
 [0.4291787 ]
 [0.43532163]
 [0.44728267]
 [0.43722054]
 [0.43486753]
 [0.42527267]
 [0.4270121 ]
 [0.4376403 ]
 [0.45185238]
 [0.44024736]
 [0.4413727 ]
 [0.43576902]
 [0.44116643]
 [0.4514633 ]
 [0.45030797]
 [0.44555753]
 [0.44297922]
 [0.43868214]
 [0.4402502 ]
 [0.45400605]
 [0.44681388]
 [0.44067687]
 [0.44321024]
 [0.44681007]
 [0.45424387]
 [0.4690951 ]
 [0.4695981 ]
 [0.46122947]
 [0.45492613]
 [0.45521846]
 [0.45416707]
 [0.4615308 ]
 [0.47572538]
 [0.46488   ]
 [0.4621686 ]
 [0.45394474]
 [0.44866648]
 [0.4566628 ]
 [0.44745204]
 [0.4387204 ]
 [0.4505889 ]
 [0.44872877]
 [0.44998372]
 [0.4568221 ]
 [0.4466896 ]
 [0.4423154 ]
 [0.43268433]
 [0.4298061 ]
 [0.43107116]
 [0.43080497]
 [0.44116694]
 [0.43466443]
 [0.4464938 ]
 [0.4541104 ]
 [0.44225234]
 [0.4434313 ]
 [0.46356785]
 [0.46121317]
 [0.45949993]
 [0.45564988]
 [0.45385644]
 [0.46193814]
 [0.45753002]
 [0.45